# Create BigQuery stored procedures

This notebook is the second of two notebooks that guide you through completing the prerequisites for running the [Real-time Item-to-item Recommendation with BigQuery ML Matrix Factorization and ScaNN](https://github.com/GoogleCloudPlatform/analytics-componentized-patterns/tree/master/retail/recommendation-system/bqml-scann) solution.

Use this notebook to create the following stored procedures that are needed by the solution:

+ `sp_ComputePMI` - Computes [pointwise mutual information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information) from item co-occurence data. This data is used by a matrix factorization model to learn item embeddings.
+ `sp_TrainItemMatchingModel` - Creates the `item_embedding_model` [matrix factorization](https://en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems)) model. This model learns item embeddings based on the PMI data computed by `sp_ComputePMI`. 
+ `sp_ExractEmbeddings` - Extracts the item embedding values from the `item_embedding_model` model, aggregates these values to produce a single embedding vector for each item, and stores these vectors in the `item_embeddings` table. The vector data is later exported to Cloud Storage to be used for item embedding lookup.

Before starting this notebook, you must run the [00_prep_bq_and_datastore](00_prep_bq_and_datastore.ipynb) notebook to complete the first part of the prerequisites.

After completing this notebook, you can run the solution either step-by-step or with a TFX pipeline:

+ To start running the solution step-by-step, run the [01_train_bqml_mf_pmi](01_train_bqml_mf_pmi.ipynb) notebook to create item embeddings.
+ To run the solution by using a TFX pipeline, run the [tfx01_interactive](tfx01_interactive.ipynb) notebook to create the pipeline.

## Setup

Install the required Python packages, configure the environment variables, and authenticate your GCP account.

In [27]:
!pip install -q -U google-cloud-bigquery pyarrow --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.18.0 requires tensorflow-io-gcs-filesystem==0.18.0, which is not installed.
tfx 1.2.0 requires google-cloud-bigquery<2.21,>=1.28.0, but you have google-cloud-bigquery 2.26.0 which is incompatible.
tfx 1.2.0 requires pyarrow<3,>=1, but you have pyarrow 5.0.0 which is incompatible.
tfx-bsl 1.2.0 requires google-cloud-bigquery<2.21,>=1.28.0, but you have google-cloud-bigquery 2.26.0 which is incompatible.
tfx-bsl 1.2.0 requires pyarrow<3,>=1, but you have pyarrow 5.0.0 which is incompatible.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.40.0 which is incompatible.
tensorflow-transform 1.2.0 requires google-cloud-bigquery<2.21,>=1.28.0, but you have google-cloud-bigquery

### Import libraries

In [28]:
import os
from google.cloud import bigquery

### Configure GCP environment settings

Update the following variables to reflect the values for your GCP environment:

+ `PROJECT_ID`: The ID of the Google Cloud project you are using to implement this solution.
+ `BUCKET`: The name of the Cloud Storage bucket you created to use with this solution. The `BUCKET` value should be just the bucket name, so `myBucket` rather than `gs://myBucket`.


In [29]:
PROJECT_ID = 'rec-ai-demo-326116' # Change to your project.
BUCKET = 'rec_bq_jsw' # Change to the bucket you created.
SQL_SCRIPTS_DIR = 'sql_scripts'
BQ_DATASET_NAME = 'css_retail'
BQ_REGION = 'US' # Change to your BigQuery region.
RESERVATION = 'default'
SLOTS=10

!gcloud config set project $PROJECT_ID

Updated property [core/project].


### Authenticate your GCP account
This is required if you run the notebook in Colab. If you use an AI Platform notebook, you should already be authenticated.

In [30]:
try:
    from google.colab import auth
    auth.authenticate_user()
    print("Colab user is authenticated.")
except: pass

## Create the stored procedure dependencies

In [44]:
%%bigquery --project $PROJECT_ID

CREATE TABLE IF NOT EXISTS css_retail.item_cooc
AS SELECT 0 AS item1_Id, 0 AS item2_Id, 0 AS cooc, 0 AS pmi;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 739.61query/s]                          


""


In [32]:
!bq mk --reservation --project_id=$PROJECT_ID --slots=$SLOTS --location=$BQ_REGION $RESERVATION

BigQuery error in mk operation: Failed to create reservation 'default': Error
reported by server with missing error fields. Server returned: {'error':
{'code': 409, 'message': 'An active reservation default already exists',
'status': 'ALREADY_EXISTS'}}


In [40]:
%%bigquery --project $PROJECT_ID

CREATE MODEL IF NOT EXISTS css_retail.item_matching_model
OPTIONS(
    MODEL_TYPE='matrix_factorization', 
    USER_COL='item1_Id', 
    ITEM_COL='item2_Id',
    RATING_COL='score'
)
AS
SELECT 0 AS item1_Id, 0 AS item2_Id, 0 AS score;


Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 2056.70query/s]                        


""


## Create the stored procedures

Run the scripts that create the BigQuery stored procedures.

In [41]:
client = bigquery.Client(project=PROJECT_ID)

In [42]:
sql_scripts = dict()

for script_file in [file for file in os.listdir(SQL_SCRIPTS_DIR) if '.sql' in file]:
  script_file_path = os.path.join(SQL_SCRIPTS_DIR, script_file)
  sql_script = open(script_file_path, 'r').read()
  sql_script = sql_script.replace('@DATASET_NAME', BQ_DATASET_NAME)
  sql_scripts[script_file] = sql_script

In [45]:
for script_file in sql_scripts:
  print(f'Executing {script_file} script...')
  query = sql_scripts[script_file]
  query_job = client.query(query)
  result = query_job.result()

print('Done.')

Executing sp_ExractEmbeddings.sql script...
Executing sp_TrainItemMatchingModel.sql script...
Executing sp_ComputePMI.sql script...
Done.


### List the stored procedures

In [46]:
query = f'SELECT * FROM {BQ_DATASET_NAME}.INFORMATION_SCHEMA.ROUTINES;'
query_job = client.query(query)
query_job.result().to_dataframe()

,specific_catalog,specific_schema,specific_name,routine_catalog,routine_schema,routine_name,routine_type,data_type,routine_body,routine_definition,external_language,is_deterministic,security_type,created,last_altered
0,rec-ai-demo-326116,css_retail,sp_ExractEmbeddings,rec-ai-demo-326116,css_retail,sp_ExractEmbeddings,PROCEDURE,None,SQL,BEGIN\n CREATE OR REPLACE TABLE css_retail.i...,None,None,None,2021-09-16 19:29:27.466000+00:00,2021-09-16 19:29:27.466000+00:00
1,rec-ai-demo-326116,css_retail,sp_ComputePMI,rec-ai-demo-326116,css_retail,sp_ComputePMI,PROCEDURE,None,SQL,BEGIN\n\n DECLARE total INT64;\n\n # Get ite...,None,None,None,2021-09-16 19:29:30.195000+00:00,2021-09-16 19:29:30.195000+00:00
2,rec-ai-demo-326116,css_retail,sp_TrainItemMatchingModel,rec-ai-demo-326116,css_retail,sp_TrainItemMatchingModel,PROCEDURE,None,SQL,BEGIN\n\n CREATE OR REPLACE MODEL css_retail....,None,None,None,2021-09-16 19:29:28.782000+00:00,2021-09-16 19:29:28.782000+00:00


You can also verify that the stored procedures have been created by viewing them in the [BigQuery console](https://pantheon.corp.google.com/bigquery).


## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**